In [1]:
!pip install gensim
!pip install python-Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 174 kB 6.7 MB/s 
     |████████████████████████████████| 2.2 MB 63.7 MB/s 


In [2]:
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz

--2022-12-26 15:49:09--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45409631 (43M) [application/x-gzip]
Saving to: ‘reviews_Cell_Phones_and_Accessories_5.json.gz’

reviews_Cell_Phones 100%[===================>]  43.31M  41.1MB/s    in 1.1s    

2022-12-26 15:49:10 (41.1 MB/s) - ‘reviews_Cell_Phones_and_Accessories_5.json.gz’ saved [45409631/45409631]



In [3]:
!gunzip reviews_Cell_Phones_and_Accessories_5.json.gz

In [4]:
import gensim
import pandas as pd

In [5]:
df = pd.read_json("/content/reviews_Cell_Phones_and_Accessories_5.json",lines=True)
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [6]:
df.shape

(194439, 9)

# Preprocessing & Tokenization

In [7]:
df['reviewText'][0]

"They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again"

In [8]:
gensim.utils.simple_preprocess("They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again")

['they',
 'look',
 'good',
 'and',
 'stick',
 'good',
 'just',
 'don',
 'like',
 'the',
 'rounded',
 'shape',
 'because',
 'was',
 'always',
 'bumping',
 'it',
 'and',
 'siri',
 'kept',
 'popping',
 'up',
 'and',
 'it',
 'was',
 'irritating',
 'just',
 'won',
 'buy',
 'product',
 'like',
 'this',
 'again']

In [9]:
review_text = df['reviewText'].apply(gensim.utils.simple_preprocess)
review_text.head()

0    [they, look, good, and, stick, good, just, don...
1    [these, stickers, work, like, the, review, say...
2    [these, are, awesome, and, make, my, phone, lo...
3    [item, arrived, in, great, time, and, was, in,...
4    [awesome, stays, on, and, looks, great, can, b...
Name: reviewText, dtype: object

# Train the Word2Vec Model

In [10]:
model = gensim.models.Word2Vec(window=10,
                               size=100,
                               min_count=2,
                               sg=1)

# Build the Vocabulory

In [11]:
model.build_vocab(review_text)

In [12]:
model.epochs

5

In [13]:
model.window

10

# Train Word2Vec Model

In [14]:
model.train(review_text, total_examples=model.corpus_count, epochs = model.epochs)

(61508286, 83868975)

In [15]:
model.corpus_count

194439

# Save the Model

In [16]:
model.save("./final_word2vec.model")

# Find the Similar Words and Similarity between the words

In [18]:
# Word Embeddings
model.wv.most_similar("bad")

[('terrible', 0.7384910583496094),
 ('horrible', 0.6944249868392944),
 ('poor', 0.6790919899940491),
 ('uncanny', 0.6573597192764282),
 ('good', 0.6561456322669983),
 ('ok', 0.6543378829956055),
 ('rediculously', 0.6512048244476318),
 ('soso', 0.6502029895782471),
 ('reveiws', 0.6497436165809631),
 ('okay', 0.6487375497817993)]

In [19]:
model.wv.similarity(w1='cheap',w2 = 'inexpensive')

0.6379401

In [20]:
matrix = model.wv.vectors # extract the word embedding matrix

print(f"shape of the word embedding matrix is {matrix.shape}")

shape of the word embedding matrix is (35561, 100)


In [21]:
W_out = model.trainables.syn1neg
W_out.shape # Output weights

(35561, 100)

In [26]:
vocab_list = list(model.wv.vocab.keys())

In [27]:
len(vocab_list)

35561

In [28]:
vocab_list

['they',
 'look',
 'good',
 'and',
 'stick',
 'just',
 'don',
 'like',
 'the',
 'rounded',
 'shape',
 'because',
 'was',
 'always',
 'bumping',
 'it',
 'siri',
 'kept',
 'popping',
 'up',
 'irritating',
 'won',
 'buy',
 'product',
 'this',
 'again',
 'these',
 'stickers',
 'work',
 'review',
 'says',
 'do',
 'on',
 'great',
 'stay',
 'phone',
 'are',
 'super',
 'stylish',
 'can',
 'share',
 'them',
 'with',
 'my',
 'sister',
 'awesome',
 'make',
 'so',
 'have',
 'only',
 'used',
 'one',
 'far',
 'had',
 'for',
 'almost',
 'year',
 'you',
 'believe',
 'that',
 'quality',
 'item',
 'arrived',
 'in',
 'time',
 'perfect',
 'condition',
 'however',
 'ordered',
 'buttons',
 'were',
 'deal',
 'included',
 'free',
 'screen',
 'protector',
 'never',
 'received',
 'though',
 'its',
 'not',
 'big',
 'would',
 've',
 'been',
 'nice',
 'to',
 'get',
 'since',
 'claim',
 'comes',
 'stays',
 'looks',
 'be',
 'multiple',
 'apple',
 'products',
 'especially',
 'having',
 'nails',
 'helps',
 'an',
 'ele

In [29]:
vocab_len = len(vocab_list)
vocab = model.wv.vocab

In [30]:
vocab

{'they': <gensim.models.keyedvectors.Vocab at 0x7fe39dc79310>,
 'look': <gensim.models.keyedvectors.Vocab at 0x7fe398e62bb0>,
 'good': <gensim.models.keyedvectors.Vocab at 0x7fe398e62a30>,
 'and': <gensim.models.keyedvectors.Vocab at 0x7fe398e629a0>,
 'stick': <gensim.models.keyedvectors.Vocab at 0x7fe398e62820>,
 'just': <gensim.models.keyedvectors.Vocab at 0x7fe398e62b50>,
 'don': <gensim.models.keyedvectors.Vocab at 0x7fe398e623d0>,
 'like': <gensim.models.keyedvectors.Vocab at 0x7fe398e62d00>,
 'the': <gensim.models.keyedvectors.Vocab at 0x7fe398e62d90>,
 'rounded': <gensim.models.keyedvectors.Vocab at 0x7fe398e620d0>,
 'shape': <gensim.models.keyedvectors.Vocab at 0x7fe398e62e20>,
 'because': <gensim.models.keyedvectors.Vocab at 0x7fe398e62880>,
 'was': <gensim.models.keyedvectors.Vocab at 0x7fe398e62ee0>,
 'always': <gensim.models.keyedvectors.Vocab at 0x7fe398e62c10>,
 'bumping': <gensim.models.keyedvectors.Vocab at 0x7fe398e62f40>,
 'it': <gensim.models.keyedvectors.Vocab at 0x

In [31]:
vocab['sum'].index

4697

In [40]:
vocab_int2word = {c.index:w for w,c in vocab.items()}

In [ ]:
vocab_int2word

In [42]:
vocab_int2word[4697]

'sum'

In [56]:
import numpy as np
one_hot = np.zeros(shape=(1,vocab_len))

one_hot[0,vocab['awesome'].index] = 1

In [57]:
z_proj = np.dot(one_hot,matrix)
z_proj.shape

(1, 100)

In [58]:
a_proj = z_proj

z_out = np.dot(a_proj,W_out.T)
z_out.shape

(1, 35561)

In [59]:
#Softmax
a_out = np.exp(z_out)/np.sum(np.exp(z_out))

In [60]:
a_out.shape

(1, 35561)

In [61]:
a_out = a_out.flatten()

indices = np.argsort(a_out) #sort in ascending order
indices = indices[::-1] # sort in descending order

for i in range(10):
  word_index = indices[i]
  word = vocab_int2word[word_index]
  a_out_i = a_out[word_index]

  print(f"{word} with score {a_out_i}")

speaker with score 0.0005438332732767601
looks with score 0.0005254425940197612
love with score 0.0004753577603815904
sound with score 0.00042167743107000865
an with score 0.00040750114709858705
design with score 0.00040097067118752803
awesome with score 0.0004001328671444776
absolutely with score 0.00039772579945447363
really with score 0.0003870661177185599
super with score 0.0003813107205999447
